## This notebook performs a single science visit for this run for a target HD 164461
### This should be run AFTER performing focus on nearby star HD 185975
##### The script, slews and acquires the target, applies focus correction, takes sequence of data

#### WARNING: This currently requires ts_externalscripts branch **tickets/DM-29061**

In [ ]:
TARGET="HD 164461" #pole

In [ ]:
import os
import sys
import asyncio
import time

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.externalscripts.auxtel.latiss_acquire_and_take_sequence import LatissAcquireAndTakeSequence

from lsst.ts.idl.enums.Script import ScriptState

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

In [ ]:
# for autocompleted to work
%config IPCompleter.use_jedi = False

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)
# if you want logging
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

# turn off logging for matplotlib
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

In [ ]:
# make sure all remotes etc are running
script = LatissAcquireAndTakeSequence(index=1)  # this essentially calls the init method of the script
#await asyncio.sleep(10) # This can be removed in the future...
await script.start_task

In [ ]:
# set wrap strategy
# this is required until the ATPtg is updated to not configure the mount for maximum time on target
script.atcs.rem.atptg.cmd_raDecTarget.set(azWrapStrategy=1)  # 1 does not unwrap, 0 unwraps

In [ ]:
# Do acquisition in the same mode we'll use to focus afterwards

In [ ]:
acq1_yaml = yaml.safe_dump({"object_name": TARGET,
                            "do_acquire": True,
                            "acq_filter" : 'RG610',
                            "acq_grating" : 'empty_1', 
                            "acq_exposure_time": 0.5,
                            "max_acq_iter": 4,
                            "target_pointing_tolerance": 2,
                            "do_pointing_model": False,
                            "dataPath": '/project/shared/auxTel/rerun/quickLook',
                            "target_pointing_verification": False})
print(acq1_yaml)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
acq1_configuration = script.cmd_configure.DataType()
acq1_configuration.config = acq1_yaml
await script.do_configure(acq1_configuration)

In [ ]:
# ATAOS must be on and corrections enabled, do as follows if required
# await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# get group_id_data that will be used for all images for this target
time_for_groupID=astropy.time.Time.now().isot
group_id_data = script.cmd_setGroupId.DataType(
                groupId=time_for_groupID
            )
await script.do_setGroupId(group_id_data)

In [ ]:
# This will align the target 
acq_results = await script.arun()

### Now perform science data with 90lpmm Ronchi

In [ ]:
seq1_yaml = yaml.safe_dump({"object_name": TARGET,
                                "do_acquire": False,
                                "do_take_sequence": True,
                                "exposure_time_sequence" : [10, 10, 10,
                                                            10, 10, 10,
                                                            10, 10, 10], 
                                "filter_sequence": ['RG610','RG610', 'RG610',
                                                    'BG40','BG40', 'BG40',
                                                    'empty_1','empty_1', 'empty_1'], 
                                # RG610 and Ronchi
                                "grating_sequence": ['ronchi90lpmm', 'ronchi90lpmm','ronchi90lpmm',
                                                     'ronchi90lpmm', 'ronchi90lpmm', 'ronchi90lpmm',
                                                     'ronchi90lpmm', 'ronchi90lpmm', 'ronchi90lpmm'],
                                "do_pointing_model": False,
                                "dataPath": '/project/shared/auxTel/rerun/quickLook',
                                "target_pointing_verification": False})
print(seq1_yaml)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
seq1_configuration = script.cmd_configure.DataType()
seq1_configuration.config = seq1_yaml
await script.do_configure(seq1_configuration)

In [ ]:
# This take the sequence of images with the Ronchi
seq1_results = await script.arun()

### Now re-acquire but using the hologram grating
##### This is the same position as the ronchi170lpmm grating

In [ ]:
acq2_yaml = yaml.safe_dump({"object_name": TARGET,
                            "do_acquire": True,
                            "acq_filter" : 'RG610',
                            "acq_grating" : 'holo4_003', 
                            "acq_exposure_time": 0.5,
                            "max_acq_iter": 4,
                            "target_pointing_tolerance": 2,
                            "do_pointing_model": False,
                            "dataPath": '/project/shared/auxTel/rerun/quickLook',
                            "target_pointing_verification": False})
print(acq2_yaml)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
acq2_configuration = script.cmd_configure.DataType()
acq2_configuration.config = acq2_yaml
await script.do_configure(acq2_configuration)

In [ ]:
# This take the acquisition sequence for the hologram (and Ronchi170lpmm grating)
acq2_results = await script.arun()

### Now perform science data with 90lpmm Ronchi

In [ ]:
seq2_yaml = yaml.safe_dump({"object_name": TARGET,
                                "do_acquire": False,
                                "do_take_sequence": True,
                                "exposure_time_sequence" : [10, 10, 10,
                                                            10, 10, 10,
                                                            10, 10, 10], 
                                "filter_sequence": ['RG610','RG610', 'RG610',
                                                    'BG40','BG40', 'BG40',
                                                    'empty_1','empty_1', 'empty_1'], 
                                # RG610 and Ronchi
                                "grating_sequence": ['ronchi90lpmm', 'ronchi90lpmm','ronchi90lpmm',
                                                     'ronchi90lpmm', 'ronchi90lpmm', 'ronchi90lpmm',
                                                     'ronchi90lpmm', 'ronchi90lpmm', 'ronchi90lpmm'],
                                "do_pointing_model": False,
                                "dataPath": '/project/shared/auxTel/rerun/quickLook',
                                "target_pointing_verification": False})
print(seq2_yaml)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
seq2_configuration = script.cmd_configure.DataType()
seq2_configuration.config = seq2_yaml
await script.do_configure(seq2_configuration)

In [ ]:
# This take the sequence of images with the Ronchi
seq2_results = await script.arun()